<a href="https://colab.research.google.com/github/bigb0ss/Retail-datasets/blob/master/Harris_Hawks_Optimization_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import scipy as spy
from time import time
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from tqdm import tqdm

In [0]:
class HHO():
  
  def __init__(self,N,T,nfeatures,pop=[]):
    self.N=N
    self.T=T
    self.n_features = nfeatures
    if len(pop)==0:
      self.population = np.random.random((self.N,self.n_features))
    else:
      self.population=pop[:]
    
    
  def optimize(self):
    
    self.UB=len(self.population)-1
    self.LB =0
    i=0
    for i in tqdm(range(self.T)):
      
      if i==0:
        print()
      best_fitness=0
      for i in range(len(self.population)):
          #model =create_model(self.population[i][0],self.population[i][1],self.population[i][2])
          acc =np.mean(self.population[i])#train_model(model)
          if acc > best_fitness:
            best_ftiness = acc
            index = i
      
      self.rabbit = self.population[index]
      
      for t,hawk in enumerate(self.population) :
        
        E=2*np.random.rand() -1
        J = 2*(1 - np.random.rand())
        
        E = self.update_E(E,i+1)
        
        if np.abs(E) >= 1:
          self.exploration(t)
        
        if np.abs(E) < 1:
          
          r=np.random.rand()
          
          if r>=0.5 and np.abs(E)>=0.5 :
            #update vector using eqn(4)
            self.soft_baise(t,E,J)
          elif r>=0.5 and np.abs(E) <0.5:
            #update vector using eqn (6)
            self.hard_baise(t,E)
          elif r<0.5 and np.abs(E) >=0.5:
            #update vector using eqn (10)
            self.soft_baise_dive(t,E,J)
          elif r<0.5 and np.abs(E) <0.5:
            #update vector using eqn (11)
            self.hard_baise_dive(t,E,J)
           
      #print("\t Best fitness value in this iteration = ",best_fitness) 
      #print("best rabbit in this iteration : ",self.rabbit)
    return self.rabbit,index

  def update_E(self,E,t):
    E = 2 * E *(1 - t/self.T)
    return E
  
  def soft_baise(self,t,E,J):
    del_x = self.rabbit - self.population[t]
    
    self.population[(t+1)%self.UB+1]= del_x  - E * np.abs( J * self.rabbit - self.population[t] )
  
  def hard_baise(self,t,E):
    del_x = self.rabbit - self.population[t]
    
    self.population[(t+1)%self.UB+1] = self.rabbit - E * np.abs(del_x)
  
  def soft_baise_dive(self,t,E,J):
    Y = self.rabbit - E* np.abs(J * self.rabbit - self.population[t])
    
    s = np.random.randn(1,self.n_features)
    
    u=np.random.random()
    v= np.random.random()
    
    std = np.std(self.population[t])
    beta = 1/1.5
    
    lf= 0.01 * u * std /(np.abs(v)**beta)
    
    Z = Y + s * lf
    
    if self.fitness(Y) < self.fitness(self.population[t]):
      self.population[(t+1)%self.UB+1] = Y
    elif self.fitness(Z) < self.fitness(self.population[t]):
      self.population[(t+1)%self.UB+1] = Z
  
  def hard_baise_dive(self,t,E,J):
    avg=0  
    for i in self.population:
      avg+=i
  
    pop = len(self.population)
    avg=avg/pop
    
    
    Y = self.rabbit - E* np.abs(J*self.rabbit - avg)
    s = np.random.randn(1,self.n_features)
    
    u=np.random.random()
    v= np.random.random()
    
    std = np.std(self.population[t])
    beta = 1/1.5
    
    lf= 0.01 * u * std /(np.abs(v)**beta)
    
    Z = Y + s * lf
    
    if self.fitness(Y) < self.fitness(self.population[t]):
      self.population[(t+1)%self.UB+1] = Y
    elif self.fitness(Z) < self.fitness(self.population[t]):
      self.population[(t+1)%self.UB+1] = Z
  
  def exploration(self,t):
    q = np.random.random()
    r1= np.random.random()
    r2= np.random.random()
    r3= np.random.random()
    r4= np.random.random()
  
    avg=0  
    for i in self.population:
      avg+=i
  
    pop = len(self.population)
    avg=avg/pop
   
    k = np.random.randint(low=0,high=pop-1)
    if q >= 0.5 :
      self.population[(t+1)%pop] = self.population[k] -r1 * (self.population[k] - 2* r2 *self.population[t])
    else :
      self.population[(t+1)%pop] = (self.rabbit - avg) - r3 *( 0 + r4 * (pop-1 - 0))
    
  def fitness(self,hawk):
    sum=np.mean(hawk)
   
    return sum

In [0]:
from sklearn.svm import LinearSVC,LinearSVR
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score

In [0]:
def create_model(c , penalty , iters):
  if np.abs(int(penalty))%2==0:
    penalty='epsilon_insensitive'
  else:
    penalty='squared_epsilon_insensitive'
  dual=True
    
  c=np.abs(c)
  iters = int(iters)
  #print(c)
  model = LinearSVR(C=c,loss=penalty,max_iter=iters,dual=dual)
  return model

In [0]:
def train_model(model):
  history = model.fit(X,y)
  return accuracy_score(y,model.predict(X))

In [7]:
train_model(create_model(2.191,0.23,-123))

NameError: ignored

In [0]:
def create_population(pop_size):
  pop=[]
  for i in range(pop_size):
    individual=[]
    n_nodes = np.random.randint(low=1,high=5)
    activation= np.random.randint(low=0,high=2)
    dropout = np.random.randint(low=1500,high=3000)
    individual.append(n_nodes)
    individual.append(activation)
    individual.append(dropout)
    pop.append(individual)
  return np.array(pop,dtype='float32')

In [9]:
create_population(5)

array([[3.000e+00, 1.000e+00, 1.805e+03],
       [4.000e+00, 1.000e+00, 2.646e+03],
       [2.000e+00, 1.000e+00, 1.969e+03],
       [3.000e+00, 1.000e+00, 1.893e+03],
       [1.000e+00, 1.000e+00, 2.167e+03]], dtype=float32)

In [0]:
pop =create_population(50)
reserve =pop[:]
#print(reserve)
opti = HHO(len(pop),1,3,pop)

In [11]:
optimal_parameters = opti.optimize()

100%|██████████| 1/1 [00:00<00:00, 76.31it/s]

In [0]:
param = reserve[optimal_parameters[1]]

In [15]:
!git clone https://github.com/bigb0ss/Retail-datasets.git
!ls

fatal: destination path 'Retail-datasets' already exists and is not an empty directory.
Retail-datasets  sample_data


In [0]:
data = pd.read_excel('Retail-datasets/Online Retail.xlsx')

In [17]:
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [18]:
print(len(data))
data.nunique()

541909


InvoiceNo      25900
StockCode       4070
Description     4223
Quantity         722
InvoiceDate    23260
UnitPrice       1630
CustomerID      4372
Country           38
dtype: int64

In [0]:
data['Price'] = data['Quantity'] * data['UnitPrice']

In [20]:
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Price
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


In [0]:
data.dropna(inplace=True)

In [0]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()

In [0]:
for i in [2,7]:
  labelencoder.fit(data.iloc[:,i])
  data.iloc[:,i]=labelencoder.transform(data.iloc[:,i])

In [24]:
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Price
0,536365,85123A,3716,6,2010-12-01 08:26:00,2.55,17850.0,35,15.30
1,536365,71053,3724,6,2010-12-01 08:26:00,3.39,17850.0,35,20.34
2,536365,84406B,861,8,2010-12-01 08:26:00,2.75,17850.0,35,22.00
3,536365,84029G,1813,6,2010-12-01 08:26:00,3.39,17850.0,35,20.34
4,536365,84029E,2776,6,2010-12-01 08:26:00,3.39,17850.0,35,20.34


In [0]:
x =data.iloc[:,[2,3,6,7]].values

In [0]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(x)

In [0]:
y=data.iloc[:,8].values

In [0]:
from sklearn.linear_model import LinearRegression

In [0]:
regressor = LinearRegression(n_jobs=5,normalize=True)

In [31]:
regressor.fit(X,y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=5, normalize=True)

In [32]:
y[0]

15.299999999999999

In [0]:
from sklearn.svm import LinearSVR

In [0]:
scr = LinearSVR()

In [35]:
scr.fit(X,y)

LinearSVR(C=1.0, dual=True, epsilon=0.0, fit_intercept=True,
          intercept_scaling=1.0, loss='epsilon_insensitive', max_iter=1000,
          random_state=None, tol=0.0001, verbose=0)

In [36]:
print('parameters : ',param)

eff_model = create_model(param[0],param[1],param[2])
eff_model.fit(X,y)

parameters :  [   2.639544     2.0104394 1313.2947   ]


LinearSVR(C=2.639544, dual=True, epsilon=0.0, fit_intercept=True,
          intercept_scaling=1.0, loss='epsilon_insensitive', max_iter=1313,
          random_state=None, tol=0.0001, verbose=0)

In [37]:
from sklearn.metrics import mean_squared_error

print("Error produced by model with Optimal parameters  : ",mean_squared_error(y,eff_model.predict(X)))

print("Error produced by model with default parameters  : ",mean_squared_error(y,scr.predict(X)))

Error produced by model with Optimal parameters  :  47125.69424677295
Error produced by model with default parameters  :  47097.77274978303


# Men-shoe-price

In [38]:
data = pd.read_csv('Retail-datasets/men-shoe-price.csv',error_bad_lines=False)

b'Skipping line 251: expected 48 fields, saw 49\nSkipping line 444: expected 48 fields, saw 50\nSkipping line 847: expected 48 fields, saw 49\nSkipping line 848: expected 48 fields, saw 49\nSkipping line 1018: expected 48 fields, saw 49\nSkipping line 1575: expected 48 fields, saw 51\nSkipping line 2133: expected 48 fields, saw 49\nSkipping line 2922: expected 48 fields, saw 51\nSkipping line 3777: expected 48 fields, saw 51\nSkipping line 4057: expected 48 fields, saw 49\nSkipping line 4239: expected 48 fields, saw 49\nSkipping line 4240: expected 48 fields, saw 49\nSkipping line 4384: expected 48 fields, saw 49\nSkipping line 4385: expected 48 fields, saw 49\nSkipping line 5388: expected 48 fields, saw 49\nSkipping line 5480: expected 48 fields, saw 49\nSkipping line 5481: expected 48 fields, saw 49\nSkipping line 5907: expected 48 fields, saw 50\nSkipping line 5908: expected 48 fields, saw 50\nSkipping line 6600: expected 48 fields, saw 49\nSkipping line 6601: expected 48 fields, sa

In [0]:
data.head()
columns=['id','brand','manufacturerNumber','name','prices.amountMin','prices.amountMax','prices.currency','prices.merchant']

In [0]:
data=data.loc[:,columns]

In [41]:
data.head()

,id,brand,manufacturerNumber,name,prices.amountMin,prices.amountMax,prices.currency,prices.merchant
0,AVpfHrJ6ilAPnD_xVXOI,Josmo,8190-W-NAVY-7.5,"Josmo 8190 Plain Infant Walking Shoes, Navy - ...",39.89,39.89,USD,NaN
1,AVpfHrJ6ilAPnD_xVXOI,Josmo,8190-W-NAVY-7.5,"Josmo 8190 Plain Infant Walking Shoes, Navy - ...",51.99,51.99,USD,UnbeatableSale - Walmart.com
2,AVpfHsWP1cnluZ0-eVZ7,SERVUS BY HONEYWELL,ZSR101BLMLG,Servus By Honeywell Shoe Studs Zsr101blmlg,40.02,40.02,USD,SIM Supply Inc - Walmart.com
3,AVpfHsWP1cnluZ0-eVZ7,SERVUS BY HONEYWELL,ZSR101BLMLG,Servus By Honeywell Shoe Studs Zsr101blmlg,50.31,50.31,USD,SIM Supply Inc - Walmart.com
4,AVpfHsWP1cnluZ0-eVZ7,SERVUS BY HONEYWELL,ZSR101BLMLG,Servus By Honeywell Shoe Studs Zsr101blmlg,46.26,46.26,USD,SIM Supply Inc - Walmart.com


In [0]:
data.dropna(inplace=True)

In [43]:
data.nunique()

id                    5605
brand                 1198
manufacturerNumber    5586
name                  5370
prices.amountMin      4844
prices.amountMax      4774
prices.currency          5
prices.merchant        548
dtype: int64

In [0]:
encode_list=[0,1,2,3,6,7]
for i in encode_list:
  labelencoder.fit(data.iloc[:,i])
  data.iloc[:,i]=labelencoder.transform(data.iloc[:,i])

In [45]:
data.head()

,id,brand,manufacturerNumber,name,prices.amountMin,prices.amountMax,prices.currency,prices.merchant
1,2045,544,2378,2097,51.99,51.99,4,421
2,2047,912,5400,4342,40.02,40.02,4,311
3,2047,912,5400,4342,50.31,50.31,4,311
4,2047,912,5400,4342,46.26,46.26,4,311
5,2047,912,5400,4342,55.99,55.99,4,311


In [0]:
X = data.iloc[:,[0,1,2,3,6,7]].values
y= data.iloc[:,5].values
X=scaler.fit_transform(X)

In [47]:
scr.fit(X,y)

LinearSVR(C=1.0, dual=True, epsilon=0.0, fit_intercept=True,
          intercept_scaling=1.0, loss='epsilon_insensitive', max_iter=1000,
          random_state=None, tol=0.0001, verbose=0)

In [48]:
eff_model.fit(X,y)

LinearSVR(C=2.639544, dual=True, epsilon=0.0, fit_intercept=True,
          intercept_scaling=1.0, loss='epsilon_insensitive', max_iter=1313,
          random_state=None, tol=0.0001, verbose=0)

In [49]:
print("Error produced by model with Optimal parameters  : ",mean_squared_error(y,eff_model.predict(X)))

print("Error produced by model with default parameters  : ",mean_squared_error(y,scr.predict(X)))

Error produced by model with Optimal parameters  :  97892.43168842036
Error produced by model with default parameters  :  98016.93296719858


# Women Shoe Price

In [50]:
data = pd.read_csv('Retail-datasets/datafiniti-womens-shoe-prices/datafiniti-womens-shoe-prices/7003_1.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (25,36,39,44) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [51]:
data.head()

,id,asins,brand,categories,colors,count,dateAdded,dateUpdated,descriptions,dimension,ean,features,flavors,imageURLs,isbn,keys,manufacturer,manufacturerNumber,merchants,name,prices.amountMin,prices.amountMax,prices.availability,prices.color,prices.condition,prices.count,prices.currency,prices.dateAdded,prices.dateSeen,prices.flavor,prices.isSale,prices.merchant,prices.offer,prices.returnPolicy,prices.shipping,prices.size,prices.source,prices.sourceURLs,prices.warranty,quantities,reviews,sizes,skus,sourceURLs,upc,websiteIDs,weight
0,AVpfBXx21cnluZ0-cKxs,NaN,Zoot,"Shoes,Clothing,Women's Shoes,All Women's Shoes","Blue,Multicolor",NaN,2016-11-11T09:49:00Z,2016-11-11T09:49:00Z,"[{""dateSeen"":[""2016-11-11T09:49:00Z""],""sourceU...",NaN,NaN,"[{""key"":""Season"",""value"":[""All-Season""]},{""key...",NaN,https://i5.walmartimages.com/asr/e04c07b2-5222...,NaN,zoot/z130103201090,Zoot,Z130103201090,"[{""dateSeen"":[""2016-11-11T09:49:00Z""],""name"":""...",Zoot Tt Trainer 2.0 Round Toe Synthetic Sne...,71.99,71.99,NaN,NaN,NaN,NaN,USD,2016-11-11T09:49:00Z,2016-11-02T00:00:00Z,NaN,TRUE,NaN,CLEARANCE USD 178.01,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Zoot-TT-TRAINER-2.0...,NaN,NaN,NaN,"10,9",NaN,https://www.walmart.com/ip/Zoot-TT-TRAINER-2.0...,NaN,NaN,NaN
1,AVpfBXx21cnluZ0-cKxs,NaN,Zoot,"Shoes,Clothing,Women's Shoes,All Women's Shoes","Blue,Multicolor",NaN,2016-11-11T09:49:00Z,2016-11-11T09:49:00Z,"[{""dateSeen"":[""2016-11-11T09:49:00Z""],""sourceU...",NaN,NaN,"[{""key"":""Season"",""value"":[""All-Season""]},{""key...",NaN,https://i5.walmartimages.com/asr/e04c07b2-5222...,NaN,zoot/z130103201090,Zoot,Z130103201090,"[{""dateSeen"":[""2016-11-11T09:49:00Z""],""name"":""...",Zoot Tt Trainer 2.0 Round Toe Synthetic Sne...,250,250,NaN,NaN,new,NaN,USD,2016-11-11T09:49:00Z,2016-10-27T00:00:00Z,NaN,FALSE,ApparelSave - Walmart.com,CLEARANCE USD 178.01,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Zoot-TT-TRAINER-2.0...,NaN,NaN,NaN,"10,9",NaN,https://www.walmart.com/ip/Zoot-TT-TRAINER-2.0...,NaN,NaN,NaN
2,AVpfBXx21cnluZ0-cKxs,NaN,Zoot,"Shoes,Clothing,Women's Shoes,All Women's Shoes","Blue,Multicolor",NaN,2016-11-11T09:49:00Z,2016-11-11T09:49:00Z,"[{""dateSeen"":[""2016-11-11T09:49:00Z""],""sourceU...",NaN,NaN,"[{""key"":""Season"",""value"":[""All-Season""]},{""key...",NaN,https://i5.walmartimages.com/asr/e04c07b2-5222...,NaN,zoot/z130103201090,Zoot,Z130103201090,"[{""dateSeen"":[""2016-11-11T09:49:00Z""],""name"":""...",Zoot Tt Trainer 2.0 Round Toe Synthetic Sne...,75.99,75.99,NaN,NaN,new,NaN,USD,2016-11-11T09:49:00Z,2016-10-27T00:00:00Z,NaN,TRUE,ApparelSave - Walmart.com,CLEARANCE USD 174.01,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Zoot-TT-TRAINER-2.0...,NaN,NaN,NaN,"10,9",NaN,https://www.walmart.com/ip/Zoot-TT-TRAINER-2.0...,NaN,NaN,NaN
3,AVpfBXx21cnluZ0-cKxs,NaN,Zoot,"Shoes,Clothing,Women's Shoes,All Women's Shoes","Blue,Multicolor",NaN,2016-11-11T09:49:00Z,2016-11-11T09:49:00Z,"[{""dateSeen"":[""2016-11-11T09:49:00Z""],""sourceU...",NaN,NaN,"[{""key"":""Season"",""value"":[""All-Season""]},{""key...",NaN,https://i5.walmartimages.com/asr/e04c07b2-5222...,NaN,zoot/z130103201090,Zoot,Z130103201090,"[{""dateSeen"":[""2016-11-11T09:49:00Z""],""name"":""...",Zoot Tt Trainer 2.0 Round Toe Synthetic Sne...,79.98,79.98,NaN,NaN,new,NaN,USD,2016-11-11T09:49:00Z,2016-11-09T00:00:00Z,NaN,TRUE,ApparelSave - Walmart.com,CLEARANCE USD 170.02,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Zoot-TT-TRAINER-2.0...,NaN,NaN,NaN,"10,9",NaN,https://www.walmart.com/ip/Zoot-TT-TRAINER-2.0...,NaN,NaN,NaN
4,AVpfEqFRLJeJML431x7w,NaN,Wild Pair,"Shoes,Women's Shoes,Clothing,All Women's Shoes",Brown,NaN,2016-11-16T12:56:36Z,2016-11-16T12:56:36Z,"[{""dateSeen"":[""2016-11-16T12:56:36Z""],""sourceU...",NaN,7.631810e+11,"[{""key"":""Heel Height"",""value"":[""High (3 in. an...",NaN,https://i5.walmartimages.com/asr/8b2c73f5-939f...,NaN,"wildpair/colfaxcog,0763181233352,763181233352",Wild Pair,COLFAX-COG,"[{""dateSeen"":[""2016-11-16T12:56:36Z""],""name"":""...",Wild Pair Colfax 

In [52]:
columns=['id','brand','manufacturerNumber','name','prices.amountMin','prices.amountMax','prices.currency','prices.merchant']
data = data.loc[:,columns]
data.head()

,id,brand,manufacturerNumber,name,prices.amountMin,prices.amountMax,prices.currency,prices.merchant
0,AVpfBXx21cnluZ0-cKxs,Zoot,Z130103201090,Zoot Tt Trainer 2.0 Round Toe Synthetic Sne...,71.99,71.99,USD,NaN
1,AVpfBXx21cnluZ0-cKxs,Zoot,Z130103201090,Zoot Tt Trainer 2.0 Round Toe Synthetic Sne...,250,250,USD,ApparelSave - Walmart.com
2,AVpfBXx21cnluZ0-cKxs,Zoot,Z130103201090,Zoot Tt Trainer 2.0 Round Toe Synthetic Sne...,75.99,75.99,USD,ApparelSave - Walmart.com
3,AVpfBXx21cnluZ0-cKxs,Zoot,Z130103201090,Zoot Tt Trainer 2.0 Round Toe Synthetic Sne...,79.98,79.98,USD,ApparelSave - Walmart.com
4,AVpfEqFRLJeJML431x7w,Wild Pair,COLFAX-COG,Wild Pair Colfax Women Peep-toe Synthetic Bro...,26.98,26.98,USD,NaN


In [53]:
data.dropna(inplace=True)
data.nunique()

id                    6348
brand                 1469
manufacturerNumber    6238
name                  6164
prices.amountMin      3073
prices.amountMax      3075
prices.currency          5
prices.merchant        428
dtype: int64

In [0]:
encode_list=[0,1,2,3,6,7]
for i in encode_list:
  labelencoder.fit(data.iloc[:,i])
  data.iloc[:,i]=labelencoder.transform(data.iloc[:,i])

In [55]:
data.head()

,id,brand,manufacturerNumber,name,prices.amountMin,prices.amountMax,prices.currency,prices.merchant
1,1938,1441,6038,6162,250,250,4,32
2,1938,1441,6038,6162,75.99,75.99,4,32
3,1938,1441,6038,6162,79.98,79.98,4,32
6,2562,1411,2828,5797,90,90,4,32
7,2638,1270,4425,5043,318,318,4,267


In [0]:
X = data.iloc[:,[0,1,2,3,6,7]].values
y= data.iloc[:,5].values
X=scaler.fit_transform(X)

In [57]:
scr.fit(X,y)

LinearSVR(C=1.0, dual=True, epsilon=0.0, fit_intercept=True,
          intercept_scaling=1.0, loss='epsilon_insensitive', max_iter=1000,
          random_state=None, tol=0.0001, verbose=0)

In [58]:
eff_model.fit(X,y)

LinearSVR(C=2.639544, dual=True, epsilon=0.0, fit_intercept=True,
          intercept_scaling=1.0, loss='epsilon_insensitive', max_iter=1313,
          random_state=None, tol=0.0001, verbose=0)

In [59]:
print("Error produced by model with Optimal parameters  : ",mean_squared_error(y,eff_model.predict(X)))

print("Error produced by model with default parameters  : ",mean_squared_error(y,scr.predict(X)))

Error produced by model with Optimal parameters  :  227577.27469832078
Error produced by model with default parameters  :  227636.07898527905
